In [1]:
import examples, core 
import os
import yaml
import dspy
import metrics

In [2]:
with open(os.path.join("..", "keys.yaml"), "r") as file:
    config = yaml.safe_load(file)
    openai_api_key = config["openai_api_key"]

llm = dspy.OpenAI(model='gpt-4o', api_key=openai_api_key, max_tokens=2000)

In [3]:
data = examples.load_examples("examples.json")

exp_metrics = metrics.Metrics(
            [
                metrics.accuracy,
                metrics.fluency,
                metrics.completeness,
                metrics.conciseness,
                metrics.context_awareness,
            ], verbose=False
        )

explingo = core.Explingo(llm=llm, context="The model predicts house prices", 
                         examples=data, metric=exp_metrics)

In [4]:
test_explanations = [d.explanation for d in data]
test_explanation_format = data[0].explanation_format

# BASIC PROMPTING

TODO: update Explingo's prompting method to take in multiple different prompt options. 
DSPy does not support this by default, but we are using DSPy's evaluations

In [5]:
explingo.run_experiment(test_explanations, test_explanation_format, prompt_type="basic", max_iters=5)

(7.212000000000001,
 accuracy             2.000
 fluency              1.000
 completeness         2.000
 conciseness          0.212
 context_awareness    2.000
 dtype: float64)

In [ ]:
explingo.run_experiment(test_explanations, test_explanation_format, prompt_type="few-shot", max_iters=5)

In [ ]:
llm.inspect_history(n=1)

In [ ]:
explingo.run_experiment(test_explanations, test_explanation_format, prompt_type="bootstrap-few-shot", max_iters=5)

In [ ]:
llm.inspect_history(n=1)